In [ ]:
import requests
import pandas as pd
import time

# URL Base da API
BASE_URL = "https://dadosabertos.camara.leg.br/api/v2"

# Definição do range de anos
ANOS_ANALISE = [2025]  # Pode ser substituído por list(range(2009, 2026))

# Função para buscar a lista de deputados
def get_deputados():
    print("Buscando a lista de deputados...", flush=True)
    url = f"{BASE_URL}/deputados"
    try:
        response = requests.get(url)
        response.raise_for_status()
        deputados = response.json().get("dados", [])
        print(f"Obtidos {len(deputados)} deputados da API.", flush=True)
        return deputados
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar API de deputados: {e}", flush=True)
        return []

# Função para buscar despesas de um deputado (sequencialmente, sem paralelização)
def get_deputado_despesas(deputado_id, anos=ANOS_ANALISE):
    despesas = []
    meses = range(1, 13)
    print(f"Iniciando coleta de despesas para deputado {deputado_id}...", flush=True)
    
    for ano in anos:
        for mes in meses:
            url = f"{BASE_URL}/deputados/{deputado_id}/despesas?ano={ano}&mes={mes}&itens=100&ordem=ASC&ordenarPor=ano"
            attempts = 5  # Número máximo de tentativas
            wait_time = 2  # Tempo inicial de espera em segundos
            
            for attempt in range(attempts):
                try:
                    response = requests.get(url)
                    if response.status_code == 429:
                        print(f"[AVISO] Erro 429 - Muitas requisições. Tentativa {attempt + 1}/{attempts}. Aguardando {wait_time} segundos...", flush=True)
                        time.sleep(wait_time)
                        wait_time *= 2  # Aumenta o tempo de espera exponencialmente
                        continue
                    response.raise_for_status()
                    data = response.json().get("dados", [])
                    time.sleep(1.5)  # Espera 1.5 segundos entre chamadas bem-sucedidas
                    if data:
                        for item in data:
                            item["deputado_id"] = deputado_id
                        print(f"✔ Obtidas {len(data)} despesas para deputado {deputado_id} em {ano}/{mes}.", flush=True)
                        despesas.extend(data)
                    else:
                        print(f"✖ Nenhuma despesa encontrada para deputado {deputado_id} em {ano}/{mes}.", flush=True)
                    break  # Sai do loop de tentativas se bem-sucedido
                except requests.exceptions.RequestException as e:
                    print(f"[ERRO] Falha ao buscar despesas do deputado {deputado_id} para {ano}/{mes}: {e}", flush=True)
                    time.sleep(wait_time)
                    wait_time *= 2
    
    print(f"✅ Finalizada coleta de despesas para deputado {deputado_id}.", flush=True)
    return despesas

# Obtendo a lista de deputados
deputados = get_deputados()

dep_despesas = []
if deputados:
    print("Iniciando coleta de despesas para todos os deputados...", flush=True)
    for deputado in deputados:
        dep_despesas.extend(get_deputado_despesas(deputado["id"]))
    
    # Convertendo para DataFrame
    despesas_df = pd.DataFrame(dep_despesas)
    
    # Criando arquivo Excel
    despesas_df.to_excel("deputados_despesas.xlsx", sheet_name="Despesas", index=False)
    
    print("🎉 Arquivo 'deputados_despesas.xlsx' salvo com sucesso!", flush=True)
else:
    print("❌ Erro ao obter a lista de deputados.", flush=True)
